## 导读

此部分包括 Action wrappers, Observation wrappers和 Reward wrappers。

# Action Wrappers

## Base Class

###  class gymnasium.ActionWrapper(env: Env[ObsType, ActType])

`gymnasium.ActionWrapper`是一个用于修改动作（action）的超类，可以在将动作传递给基础环境的`step()`方法之前进行修改。如果你希望在动作被传递给环境之前应用某个函数，你可以通过继承`ActionWrapper`并重写`action()`方法来实现这个转换。在`action()`方法中定义的转换必须接受基础环境动作空间中的值。然而，其定义域可能与原始动作空间不同。在这种情况下，你需要通过在包装器的`__init__()`方法中设置`action_space`来指定新的动作空间。

Gymnasium提供了如`gymnasium.wrappers.ClipAction`和`gymnasium.wrappers.RescaleAction`等动作包装器，用于裁剪和缩放动作。

### 参数
- **env** (`Env`): 要包装的环境。

### 方法
- **action(action: WrapperActType) → ActType**:
  - **参数**:
    - **action**: 原始的`step()`方法中的动作。
  - **返回**: 修改后的动作。

### 使用示例
下面是一个简单的示例，展示如何创建一个自定义的动作包装器，将所有动作的数值加倍后再传递给环境：
```python
import gymnasium as gym
from gymnasium import spaces

class DoubleAction(gym.ActionWrapper):
    def __init__(self, env):
        super(DoubleAction, self).__init__(env)
        # 假设原始环境的动作空间是连续的（Box）
        self.action_space = spaces.Box(low=env.action_space.low*2, high=env.action_space.high*2, shape=env.action_space.shape, dtype=env.action_space.dtype)

    def action(self, action):
        # 将动作的每个维度值加倍
        return action * 2

# 创建一个环境并应用DoubleAction包装器
env = gym.make("CartPole-v1")
env = DoubleAction(env)

# 重置环境并采取一个动作
obs = env.reset()
action = env.action_space.sample()  # 从新的动作空间采样
obs, reward, done, info = env.step(action)  # 执行加倍后的动作
```
在这个示例中，`DoubleAction`包装器将所有传递给环境的动作值加倍，这对于调整动作空间范围或实验不同的动作策略等情况非常有用。通过继承`ActionWrapper`，你可以轻松地对动作进行各种自定义修改。

### 另一个例子：

假设我们正在开发一个强化学习环境，其中的代理需要操作一个机械臂去抓取物体。原始环境接受的是连续动作空间中的动作，表示机械臂的每个关节旋转角度。为了简化问题，我们决定将动作空间离散化，让代理只能选择预定义的一组动作来控制机械臂。我们将创建一个自定义的动作包装器`DiscretizeActionWrapper`，它将连续的动作空间映射到离散的动作空间，并在执行动作之前将离散动作转换回连续动作。

### 自定义动作包装器：DiscretizeActionWrapper

```python
import numpy as np
import gymnasium as gym
from gymnasium import spaces

class DiscretizeActionWrapper(gym.ActionWrapper):
    def __init__(self, env, num_actions=5):
        super(DiscretizeActionWrapper, self).__init__(env)
        self.num_actions = num_actions
        # 假设原始环境的动作空间是一维连续的（Box），这里我们将其离散化
        self.action_space = spaces.Discrete(num_actions)

        # 预定义的动作集，这里简化为等间隔地选择动作空间中的值
        self.action_set = np.linspace(env.action_space.low[0], env.action_space.high[0], num_actions)

    def action(self, action):
        # 将离散动作转换回连续动作
        continuous_action = np.array([self.action_set[action]])
        return continuous_action

# 创建一个原始环境
env = gym.make("Pendulum-v1")
# 应用DiscretizeActionWrapper
env = DiscretizeActionWrapper(env)

# 重置环境并采取一个动作
obs = env.reset()
for _ in range(10):
    # 从离散的动作空间采样
    action = env.action_space.sample()
    print(f"Selected discrete action: {action}")

    # 执行转换后的连续动作
    obs, reward, done, info = env.step(action)
    print(f"Performed continuous action: {env.action_set[action]}")
```

在这个例子中，`DiscretizeActionWrapper`将原始环境的连续动作空间离散化为5个动作。每个动作对应于机械臂关节旋转角度的一种预设值。当代理选择一个离散动作时，包装器将其转换为对应的连续动作，然后传递给原始环境执行。这种方法简化了代理的决策过程，使其更容易学习如何控制机械臂，同时仍然保持了一定程度的控制精度。

## 其他可用的动作包装器 Available Action Wrappers

### class gymnasium.wrappers.TransformAction(env: gym.Env[ObsType, ActType], func: Callable[[WrapperActType], ActType], action_space: Space[WrapperActType] | None)

`gymnasium.wrappers.TransformAction`是一个包装器，用于在将动作传递给环境的`step()`函数之前对其应用一个函数。这个功能允许用户对动作进行自定义的转换或修改，例如缩放、偏移或其他任何需要的转换，以适应环境的特定要求或实验不同的策略。

### 参数
- **env** (`gym.Env`): 要包装的环境。
- **func** (`Callable[[WrapperActType], ActType]`): 应用于`step()`方法动作的函数。这个函数接受一个动作作为输入，并返回修改后的动作。
- **action_space** (`Space[WrapperActType]` 或 `None`): 给定转换函数后，包装器的新动作空间。如果提供，这将替代原始环境的动作空间。

### 使用场景
- **动作缩放**：将动作值缩放到不同的范围，以适应环境的特定要求。
- **动作偏移**：对动作值进行偏移，可能用于实验不同的控制策略。
- **动作转换**：将动作从一种格式转换为环境能够接受的另一种格式，例如，从离散动作到连续动作的转换。

### 示例代码
下面的示例展示了如何使用`TransformAction`包装器将动作的值缩小一半并加上0.1的偏移，然后传递给环境：

```python
import numpy as np
import gymnasium as gym

# 创建环境
env = gym.make("MountainCarContinuous-v0")

# 定义动作转换函数：将动作值缩小一半并加上0.1的偏移
def transform_action(a):
    return 0.5 * a + 0.1

# 应用TransformAction包装器
env = gym.wrappers.TransformAction(env, transform_action, env.action_space)

# 重置环境
_ = env.reset(seed=123)

# 执行转换后的动作
obs, *_= env.step(np.array([0.0, 1.0]))

# 打印观察结果
print(obs)
```

在这个例子中，`TransformAction`包装器使得动作在传递给`step()`方法之前先被转换。这种方法可以在不修改原始环境代码的情况下，灵活地实验不同的动作处理策略。



通过使用`TransformAction`包装器，用户可以非常灵活地控制如何修改和处理传递给环境的动作，这对于实验和优化强化学习策略非常有帮助。

### class gymnasium.wrappers.ClipAction(env: Env[ObsType, ActType])

`gymnasium.wrappers.ClipAction`是一个包装器，用于在将动作传递给环境的`step()`方法之前对其进行裁剪，确保动作位于环境的动作空间范围内。这个包装器非常实用，特别是在动作可能超出环境预期范围的情况下，它可以自动将动作调整到有效范围，防止可能的错误或异常行为。

### 示例代码
```python
import gymnasium as gym
from gymnasium.wrappers import ClipAction
import numpy as np

# 创建一个环境
env = gym.make("Hopper-v4", disable_env_checker=True)

# 应用ClipAction包装器
env = ClipAction(env)

# 打印动作空间，注意到动作空间可能是无限的，但实际执行的动作会被裁剪到有效范围
print(env.action_space)  # 输出：Box(-inf, inf, (3,), float32)

# 重置环境
_ = env.reset(seed=42)

# 尝试执行一个超出动作空间的动作
_ = env.step(np.array([5.0, -2.0, 0.0], dtype=np.float32))
# 实际执行的动作将被裁剪到环境动作空间的有效范围内
```

### 参数
- **env** (`Env`): 要包装的环境。


通过使用`ClipAction`包装器，可以简化处理动作空间限制的逻辑，确保所有传递给环境的动作都在有效范围内，从而避免潜在的问题。此外，这个包装器还提供了一个向量化版本`gymnasium.wrappers.vector.ClipAction`，可以用于同时处理多个环境实例，进一步提高了处理效率和灵活性。

### class gymnasium.wrappers.RescaleAction(env: gym.Env[ObsType, ActType], min_action: float | int | np.ndarray, max_action: float | int | np.ndarray)

`gymnasium.wrappers.RescaleAction`是一个包装器，用于在将动作传递给环境的`step()`方法之前，线性地将Box类型的动作空间重新缩放到指定的[min_action, max_action]范围内。这允许开发者调整动作的范围，以适应环境或模型的特定需求，无需修改环境本身的代码。

### 示例代码
```python
import gymnasium as gym
from gymnasium.wrappers import RescaleAction
import numpy as np

# 创建一个环境
env = gym.make("Hopper-v4", disable_env_checker=True)

# 重置环境并执行一组动作
_ = env.reset(seed=42)
obs, _, _, _, _ = env.step(np.array([1, 1, 1], dtype=np.float32))

# 重新设置环境，应用RescaleAction包装器来调整动作空间的范围
_ = env.reset(seed=42)
min_action = -0.5
max_action = np.array([0.0, 0.5, 0.75], dtype=np.float32)
wrapped_env = RescaleAction(env, min_action=min_action, max_action=max_action)

# 在包装环境中执行动作，并验证观察结果是否与原始环境一致
wrapped_env_obs, _, _, _, _ = wrapped_env.step(max_action)
assert np.all(obs == wrapped_env_obs), "The observations should be equal."

```

### 参数
- **env** (`Env`): 要包装的环境。
- **min_action** (`float`, `int` 或 `np.ndarray`): 每个动作的最小值。这可以是一个numpy数组或一个标量。
- **max_action** (`float`, `int` 或 `np.ndarray`): 每个动作的最大值。这可以是一个numpy数组或一个标量。


`RescaleAction`包装器使得在不改变原始环境动作空间定义的情况下，通过简单地包装环境即可实现动作范围的调整。这对于那些需要将环境集成到具有不同动作范围需求的强化学习模型中的开发者来说，是一个非常有用的工具。此外，这个包装器还提供了一个向量化版本`gymnasium.wrappers.vector.RescaleAction`，支持同时处理多个环境实例。

### class gymnasium.wrappers.StickyAction(env: Env[ObsType, ActType], repeat_action_probability: float)

`gymnasium.wrappers.StickyAction`是一个包装器，用于在环境中添加动作重复的概率。这意味着在每次调用`step()`方法时，有一定的概率（由`repeat_action_probability`参数指定）当前的动作会被重复执行，而不是执行新的动作。这种行为模仿了某些现实世界场景中的不确定性，比如控制器的滞后或响应不一致等问题。

### 示例代码
```python
import gymnasium as gym
from gymnasium.wrappers import StickyAction

# 创建环境并应用StickyAction包装器
env = gym.make("CartPole-v1")
env = StickyAction(env, repeat_action_probability=0.9)

# 重置环境
obs, info = env.reset(seed=123)
print(obs)  # 打印初始观察

# 执行一系列动作，注意到由于StickyAction的效果，某些动作可能会被重复执行
for _ in range(5):
    action = env.action_space.sample()  # 随机选择一个动作
    obs, reward, done, truncated, info = env.step(action)  # 执行动作
    print(obs, reward, done, truncated)  # 打印观察结果和奖励
```

### 参数
- **env** (`Env`): 要包装的环境。
- **repeat_action_probability** (`float`): 重复前一个动作的概率。这个值应该在0到1之间，表示重复动作的概率。


通过使用`StickyAction`包装器，开发者可以模拟动作执行中的不确定性，这对于测试和训练更加健壮和适应性强的强化学习模型非常有用。该包装器按照Machado等人在2018年提出的实现进行实现，特别适合于那些需要考虑动作执行不确定性的场景。

# Observation Wrappers

### class gymnasium.ObservationWrapper(env: Env[ObsType, ActType])

`gymnasium.ObservationWrapper`是Gymnasium库中的一个包装器，用于修改环境的观察值。这个包装器允许开发者在观察值被传递给学习算法之前应用一个自定义的转换函数。通过继承`ObservationWrapper`并重写`observation()`方法，可以实现观察值的转换。这种转换可以包括归一化、重塑观察空间、转换数据类型、添加特征等操作。

### 参数
- **env** (`Env`): 要包装的环境。

### 方法
- **observation(observation: ObsType) → WrapperObsType**:
  - **参数**:
    - **observation**: 环境产生的原始观察值。
  - **返回**: 修改后的观察值。

### 使用示例

假设我们有一个环境，其观察值是一个图像（例如，一个`210x160x3`的数组，代表一个带有RGB通道的210x160像素图像）。现在，我们想要将这个观察值转换为灰度图像，并将其大小重塑为`84x84`，以减少模型的输入维度。

```python
import gymnasium as gym
from gymnasium.wrappers import ObservationWrapper
import cv2
import numpy as np

class ResizeAndGrayscaleObservation(ObservationWrapper):
    def __init__(self, env):
        super(ResizeAndGrayscaleObservation, self).__init__(env)
        # 设置新的观察空间维度
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(84, 84), dtype=np.uint8)

    def observation(self, observation):
        # 将观察值转换为灰度并重塑大小
        observation = cv2.cvtColor(observation, cv2.COLOR_RGB2GRAY)
        observation = cv2.resize(observation, (84, 84), interpolation=cv2.INTER_AREA)
        return observation

# 创建环境并应用自定义观察包装器
env = gym.make("Pong-v0")
env = ResizeAndGrayscaleObservation(env)

# 重置环境并获得转换后的观察值
observation = env.reset()
print("Transformed Observation Shape:", observation.shape)
```

通过使用`ObservationWrapper`，开发者可以轻松地对环境产生的观察值进行必要的转换，以适应不同的学习模型或实验设置，而不需要修改环境本身的代码。这提高了代码的复用性和灵活性。

## Implemented Wrappers


### class gymnasium.wrappers.TransformObservation(env: gym.Env[ObsType, ActType], func: Callable[[ObsType], Any], observation_space: gym.Space[WrapperObsType] | None)


`gymnasium.wrappers.TransformObservation`是一个包装器，用于在将观察值从环境的`Env.reset()`和`Env.step()`返回给用户之前，对其应用一个自定义的函数`func`。这可以用于观察值的预处理，比如特征工程、数据归一化、添加噪声等。

### 示例代码
```python
import gymnasium as gym
from gymnasium.wrappers import TransformObservation
import numpy as np

np.random.seed(0)

# 创建环境
env = gym.make("CartPole-v1")

# 重置环境并获取原始观察值
obs, _ = env.reset(seed=42)
print("Original observation:", obs)

# 创建并应用TransformObservation包装器
# 添加随机噪声到观察值
env = TransformObservation(env, lambda obs: obs + 0.1 * np.random.random(obs.shape))

# 重置包装后的环境并获取转换后的观察值
transformed_obs, _ = env.reset(seed=42)
print("Transformed observation:", transformed_obs)
```

### 参数
- **env** (`Env`): 要包装的环境。
- **func** (`Callable[[ObsType], Any]`): 将被应用到所有观察值的转换函数。如果`func`返回的观察值超出了原始环境观察空间的界限，那么应该提供一个更新后的`observation_space`。
- **observation_space** (`gym.Space[WrapperObsType]` 或 `None`): 包装器的观察空间。如果为`None`，则假定与原始环境的观察空间相同。


`TransformObservation`包装器使得在不修改环境内部代码的情况下，对环境产生的观察值进行自定义处理变得简单快捷。这个功能对于实验不同的观察值处理方法，或者对观察值进行必要的预处理以适应特定的学习算法非常有用。此外，这个包装器还提供了一个向量化版本`gymnasium.wrappers.vector.TransformObservation`，支持对向量化环境中的观察值进行转换。

### class gymnasium.wrappers.DelayObservation(env: Env[ObsType, ActType], delay: int)

`gymnasium.wrappers.DelayObservation`是一个包装器，用于在环境返回的观察值中添加延迟。这意味着，在达到指定的延迟步数之前，环境将返回一个与观察空间形状相同的零数组。这种设置模拟了现实世界中可能遇到的观察延迟现象，例如，从传感器接收数据的延迟。

### 示例代码
```python
import gymnasium as gym
from gymnasium.wrappers import DelayObservation

# 创建环境
env = gym.make("CartPole-v1")

# 重置环境并获取初始观察值
obs, _ = env.reset(seed=123)
print("Initial observation without delay:", obs)

# 应用DelayObservation包装器并设置延迟为2步
env = DelayObservation(env, delay=2)

# 重置包装后的环境并获取延迟后的观察值
delayed_obs, _ = env.reset(seed=123)
print("Initial observation with delay:", delayed_obs)

# 执行两步动作，观察返回的观察值
for _ in range(2):
    delayed_obs, _, _, _, _ = env.step(env.action_space.sample())
    print("Delayed observation:", delayed_obs)
```

### 参数
- **env** (`Env`): 要包装的环境。
- **delay** (`int`): 观察值的延迟步数。在这些步数之前，环境将返回零数组作为观察值。

### 注意
- 这个包装器不支持随机延迟值。如果用户对此功能感兴趣，请在Gymnasium的GitHub仓库中提出问题或提交拉取请求。


通过使用`DelayObservation`包装器，开发者可以模拟环境中的观察延迟，这对于测试强化学习算法在面对不完美信息时的鲁棒性非常有用。这种设置特别适用于模拟现实世界场景，如无人驾驶汽车或机器人控制，其中传感器数据的延迟可能对系统性能产生重大影响。

### class gymnasium.wrappers.DtypeObservation(env: Env[ObsType, ActType], dtype: Any)


`gymnasium.wrappers.DtypeObservation` 是一个包装器，用于将环境观察值数组的数据类型修改为指定的`dtype`。这个包装器特别有用于那些需要将环境观察值的数据类型与模型输入数据类型对齐的情况，例如，将观察值从默认的`float64`转换为`float32`以减少计算资源消耗或与特定机器学习框架的数据类型要求相匹配。

### 注意
- 这个包装器仅与`Box`, `Discrete`, `MultiDiscrete` 和 `MultiBinary` 观察空间兼容。

### 示例代码
```python
import gymnasium as gym
from gymnasium.wrappers import DtypeObservation

# 创建环境
env = gym.make("CartPole-v1")

# 查看原始观察空间的数据类型
print("Original observation dtype:", env.observation_space.dtype)

# 应用DtypeObservation包装器并设置新的数据类型为float32
env = DtypeObservation(env, dtype='float32')

# 查看修改后的观察空间的数据类型
print("Modified observation dtype:", env.observation_space.dtype)

# 重置环境并获取观察值
obs, _ = env.reset(seed=42)
print("Observation dtype after reset:", obs.dtype)
```

### 参数
- **env** (`Env`): 要包装的环境。
- **dtype** (`Any`): 观察值数组的新数据类型。

通过使用`DtypeObservation`包装器，开发者可以确保环境观察值的数据类型满足特定的需求，进而提高数据处理的效率和模型训练的兼容性。此外，这个包装器还提供了一个向量化版本`gymnasium.wrappers.vector.DtypeObservation`，支持对向量化环境中的观察值进行数据类型转换。

### class gymnasium.wrappers.FilterObservation(env: gym.Env[ObsType, ActType], filter_keys: Sequence[str | int])


`gymnasium.wrappers.FilterObservation` 是一个包装器，用于过滤 `Dict` 或 `Tuple` 观察空间中的特定子空间，只保留一组指定的键（对于 `Dict`）或索引（对于 `Tuple`）。这对于在观察空间中只关注部分信息时非常有用，可以帮助减少模型的输入维度，专注于重要的特征。

### 示例代码
```python
import gymnasium as gym
from gymnasium.wrappers import FilterObservation, TimeAwareObservation

# 创建环境并添加时间感知观察
env = gym.make("CartPole-v1")
env = TimeAwareObservation(env, flatten=False)

# 显示添加时间感知后的观察空间
print("Observation space after adding time awareness:", env.observation_space)

# 应用FilterObservation包装器，只保留"time"键
env = FilterObservation(env, filter_keys=['time'])

# 重置环境并查看过滤后的观察值
obs, _ = env.reset(seed=42)
print("Filtered observation after reset:", obs)

# 执行动作并查看过滤后的观察值
obs, _, _, _, _ = env.step(0)
print("Filtered observation after step:", obs)
```

### 参数
- **env** (`Env`): 要包装的环境。
- **filter_keys** (`Sequence[str | int]`): 要包含的子空间的键集合。对于 `Dict` 观察空间使用字符串列表，对于 `Tuple` 观察空间使用整数列表。

### 更改日志
- `v0.12.3`: 最初添加，最初命名为 `FilterObservationWrapper`。
- `v1.0.0`: 重命名为 `FilterObservation` 并添加对 `Tuple` 观察空间的支持，使用整数 `filter_keys`。

通过使用 `FilterObservation` 包装器，开发者可以简化观察空间，去除不必要的信息，从而减轻学习算法的负担。此外，这个包装器还提供了一个向量化版本 `gymnasium.wrappers.vector.FilterObservation`，支持对向量化环境中的观察值进行过滤。

### class gymnasium.wrappers.FlattenObservation(env: Env[ObsType, ActType])

`gymnasium.wrappers.FlattenObservation` 是一个包装器，用于平展化环境的观察空间以及来自`reset`和`step`函数的每个观察值。这个包装器特别适用于将高维的、结构化的观察空间（如图像、多维数组或字典）转换成一维数组，以便简化数据处理和模型输入。

### 示例代码
```python
import gymnasium as gym
from gymnasium.wrappers import FlattenObservation

# 创建环境
env = gym.make("CarRacing-v2")

# 查看原始观察空间的形状
print("Original observation space shape:", env.observation_space.shape)

# 应用FlattenObservation包装器
env = FlattenObservation(env)

# 查看平展化后的观察空间的形状
print("Flattened observation space shape:", env.observation_space.shape)

# 重置环境并获取平展化后的观察值
obs, _ = env.reset()
print("Shape of flattened observation after reset:", obs.shape)
```

### 参数
- **env** (`Env`): 要包装的环境。


通过使用`FlattenObservation`包装器，开发者可以将复杂的观察空间转换为简单的一维数组形式，从而降低数据预处理的复杂性并提高模型处理的效率。此外，这个包装器还提供了一个向量化版本`gymnasium.wrappers.vector.FlattenObservation`，支持对向量化环境中的观察值进行平展化处理。

### class gymnasium.wrappers.FrameStackObservation(env: gym.Env[ObsType, ActType], stack_size: int, *, padding_type: str | ObsType = 'reset')

`gymnasium.wrappers.FrameStackObservation` 是一个包装器，用于将最近 N 个时间步的观察值堆叠在一起。这种方法对于那些需要观察值中的时间序列信息以进行决策的场景特别有用，例如，在视频游戏或序列预测任务中。

### 示例代码
```python
import gymnasium as gym
from gymnasium.wrappers import FrameStackObservation
import numpy as np

# 创建环境并应用FrameStackObservation包装器
env = gym.make("CarRacing-v2")
env = FrameStackObservation(env, stack_size=4)

# 查看堆叠后的观察空间
print("Stacked observation space:", env.observation_space)

# 重置环境并获取堆叠后的观察值
obs, _ = env.reset()
print("Shape of stacked observation after reset:", obs.shape)

# 使用不同的填充观察选项
env = gym.make("CartPole-v1")
stacked_env = FrameStackObservation(env, 3, padding_type="zero")
obs, _ = stacked_env.reset(seed=123)
print("Zero-padded stacked observation shape:", obs.shape)

stacked_env = FrameStackObservation(env, 3, padding_type=np.array([1, -1, 0, 2], dtype=np.float32))
obs, _ = stacked_env.reset(seed=123)
print("Custom-padded stacked observation:", obs)
```

### 参数
- **env** (`Env`): 要包装的环境。
- **stack_size** (`int`): 要堆叠的帧数。
- **padding_type** (`str | ObsType`): 堆叠观察值时使用的填充类型。选项包括"reset"（默认，重置值重复）、"zero"（使用零填充）和自定义观察值实例。

### 更改日志
- `v0.15.0`: 最初添加为`FrameStack`，支持 lz4 压缩。
- `v1.0.0`: 重命名为`FrameStackObservation`，移除 lz4 和 LazyFrame 支持，同时添加`padding_type`参数。

通过使用`FrameStackObservation`包装器，开发者可以在模型输入中包含多个时间步的信息，从而使模型能够基于时间序列数据做出更准确的预测或决策。

### class gymnasium.wrappers.GrayscaleObservation(env: Env[ObsType, ActType], keep_dim: bool = False)


`gymnasium.wrappers.GrayscaleObservation` 是一个用于将环境中的图像观察值从 RGB 转换为灰度图的包装器。这个转换有助于减少处理图像数据所需的计算资源，同时保持对环境状态的关键信息。

### 示例代码
```python
import gymnasium as gym
from gymnasium.wrappers import GrayscaleObservation

# 创建环境
env = gym.make("CarRacing-v2")

# 查看原始观察空间形状
print("Original observation space shape:", env.observation_space.shape)

# 应用GrayscaleObservation包装器，不保留通道维度
grayscale_env = GrayscaleObservation(env)
print("Grayscale observation space shape without keeping dimension:", grayscale_env.observation_space.shape)

# 应用GrayscaleObservation包装器，保留通道维度
grayscale_env_keep_dim = GrayscaleObservation(env, keep_dim=True)
print("Grayscale observation space shape with keeping dimension:", grayscale_env_keep_dim.observation_space.shape)
```

### 参数
- **env** (`Env`): 要包装的环境。
- **keep_dim** (`bool`): 是否保留通道维度。如果设置为`True`，观察值将保持三维形状（高度，宽度，1）；否则，观察值将是二维的（高度，宽度）。

### 更改日志
- `v0.15.0`: 最初添加为`GrayScaleObservation`。
- `v1.0.0`: 更名为`GrayscaleObservation`。

通过使用`GrayscaleObservation`包装器，开发者可以简化图像处理流程并减少模型训练和推断所需的计算资源，尤其在图像数据占主导地位的环境中。此外，这个包装器提供了灵活性，允许开发者根据自己的需要选择是否保留通道维度。

### class gymnasium.wrappers.MaxAndSkipObservation(env: Env[ObsType, ActType], skip: int = 4)

`gymnasium.wrappers.MaxAndSkipObservation` 是一个包装器，用于在环境中跳过指定数量的帧（观察值），并在跳过的帧中选择最大值作为最终的观察值返回。这种方法主要用于降低决策频率，并通过考虑连续观察值的最大值来减少可能的视觉闪烁影响，这在某些游戏环境中可能会对性能产生显著影响。

### 示例代码
```python
import gymnasium as gym
import numpy as np

# 创建环境
env = gym.make("CartPole-v1")

# 对连续4个动作的观察值进行手动计算
obs0, *_ = env.reset(seed=123)
obs1, *_ = env.step(1)
obs2, *_ = env.step(1)
obs3, *_ = env.step(1)
obs4, *_ = env.step(1)

# 计算最后两个观察值的最大值
skip_and_max_obs = np.max(np.stack([obs3, obs4], axis=0), axis=0)

# 应用 MaxAndSkipObservation 包装器
wrapped_env = MaxAndSkipObservation(env, skip=4)
wrapped_obs0, *_ = wrapped_env.reset(seed=123)

# 执行一个动作，观察结果
wrapped_obs1, *_ = wrapped_env.step(1)

# 验证原始观察值与包装器处理后的观察值是否相同
print(np.all(obs0 == wrapped_obs0))  # 应为 True
print(np.all(wrapped_obs1 == skip_and_max_obs))  # 应为 True
```

### 参数
- **env** (`Env`): 要应用包装器的环境。
- **skip** (`int`): 需要跳过的帧数。默认为4，即在每个动作之后跳过三个帧，然后返回第四个帧和第三个帧的最大值。

通过使用 `MaxAndSkipObservation` 包装器，开发者可以在不牺牲太多决策质量的情况下，有效降低环境中的决策频率。这种方法在处理那些具有高时间冗余性的环境时尤其有用，如某些 Atari 游戏。

### class gymnasium.wrappers.NormalizeObservation(env: Env[ObsType, ActType], epsilon: float = 1e-8)


`gymnasium.wrappers.NormalizeObservation` 是一个用于标准化环境观察值的包装器，它可以将观察值调整为以均值为中心、单位方差的分布。这对于那些观察值具有不同量级或分布的环境特别有用，因为它可以帮助学习算法更快地收敛。

### 示例代码
```python
import numpy as np
import gymnasium as gym

# 创建环境并获取一次观察值，用于展示未标准化前的观察值
env = gym.make("CartPole-v1")
obs, info = env.reset(seed=123)
term, trunc = False, False
while not (term or trunc):
    obs, _, term, trunc, _ = env.step(1)

print("Observation before normalization:", obs)

# 应用 NormalizeObservation 包装器，并再次获取观察值，用于展示标准化后的观察值
env = gym.make("CartPole-v1")
env = NormalizeObservation(env)
obs, info = env.reset(seed=123)
term, trunc = False, False
while not (term or trunc):
    obs, _, term, trunc, _ = env.step(1)

print("Observation after normalization:", obs)
```

### 参数
- **env** (`Env`): 要应用包装器的环境。
- **epsilon** (`float`): 一个稳定性参数，用于在缩放观察值时防止除以0。默认值为 `1e-8`。

### 注意事项
- 观察值的标准化依赖于历史轨迹，如果包装器是新实例化的，或者最近更改了策略，则观察值可能无法正确标准化。
- 通过 `update_running_mean` 属性，可以在评估时禁用更新运行均值/标准差，这对于使用先前计算的统计数据而不是实时更新它们特别有用。

该包装器适用于那些观察值分布广泛或在不同尺度上的环境，有助于提高学习效率和稳定性。

### class gymnasium.wrappers.AddRenderObservation(env: Env[ObsType, ActType], render_only: bool = True, render_key: str = 'pixels', obs_key: str = 'state')

`gymnasium.wrappers.AddRenderObservation` 包装器可以将环境的渲染观测值（例如，RGB图像）包含在环境的观测值中。这对于需要同时处理原始环境状态和渲染图像的情况特别有用。

### 示例代码

**替换观测值为渲染图像**:
```python
import gymnasium as gym
import numpy as np

# 创建环境并应用包装器，仅返回渲染图像作为观测值
env = gym.make("CartPole-v1", render_mode="rgb_array")
env = AddRenderObservation(env, render_only=True)
print("Observation space after adding render:", env.observation_space)

# 重置环境并比较观测值和渲染图像
obs, _ = env.reset(seed=123)
image = env.render()
assert np.all(obs == image), "The observation should be the rendered image"

# 执行动作并再次比较
obs, *_ = env.step(env.action_space.sample())
image = env.render()
assert np.all(obs == image), "The observation should be the rendered image after step"
```

**将渲染图像添加到原始观测值**:
```python
# 创建环境并应用包装器，同时保留原始观测值和渲染图像
env = gym.make("CartPole-v1", render_mode="rgb_array")
env = AddRenderObservation(env, render_only=False)
print("Observation space after adding render:", env.observation_space)

# 重置环境并检查观测值字典的键
obs, info = env.reset(seed=123)
print("Keys in observation:", obs.keys())

# 检查原始状态和渲染图像是否正确包含在观测值中
assert "state" in obs and "pixels" in obs, "Observation should contain 'state' and 'pixels'"
assert np.all(obs["pixels"] == env.render()), "The 'pixels' key should contain the rendered image"
```

### 参数解读
- **env** (`Env`): 要包装的环境实例。
- **render_only** (`bool`): 如果为`True`，则只返回渲染图像作为观测值；如果为`False`，则在观测值字典中既包含原始观测值也包含渲染图像。
- **render_key** (`str`): 用于指定包含渲染图像的键名，默认为`"pixels"`。
- **obs_key** (`str`): 用于指定包含原始观测值的键名，默认为`"state"`。

### 注意事项
- 当`render_only=True`时，原始观测值将被丢弃，只保留渲染图像作为观测值。
- 当`render_only=False`时，观测值将是一个字典，包含原始状态（`"state"`键）和渲染图像（`"pixels"`键）。

### class gymnasium.wrappers.ResizeObservation(env: Env[ObsType, ActType], shape: tuple[int, int])

`gymnasium.wrappers.ResizeObservation` 包装器允许调整环境观测值图像的大小。这是通过使用 OpenCV 库来完成的，可以将图像重新缩放到指定的维度。这个包装器特别适合于需要将图像输入到模型中但图像尺寸不匹配的情况。

### 示例代码

```python
import gymnasium as gym
from gymnasium.wrappers import ResizeObservation

# 创建环境并应用包装器以调整观测图像的大小
env = gym.make("CarRacing-v2")
print("Original observation space shape:", env.observation_space.shape)

# 应用 ResizeObservation 包装器以将观测图像大小调整为 32x32
resized_env = ResizeObservation(env, (32, 32))
print("Resized observation space shape:", resized_env.observation_space.shape)

# 重置环境并获取调整大小后的观测值
obs, _ = resized_env.reset()
print("Shape of resized observation:", obs.shape)
```

### 参数解读
- **env** (`Env`): 要包装的环境实例。
- **shape** (`tuple[int, int]`): 调整后的图像观测值的大小，必须提供为两个整数的元组，表示新的图像高度和宽度。

### 注意事项
- 这个包装器通过使用 OpenCV 库来调整图像的大小，因此需要确保已安装了 `opencv-python` 包。
- `shape` 参数指定了调整后的图像尺寸，通常用于将图像尺寸标准化以适应深度学习模型的输入要求。

### class gymnasium.wrappers.ReshapeObservation(env: gym.Env[ObsType, ActType], shape: int | tuple[int, ...])

`gymnasium.wrappers.ReshapeObservation` 包装器允许将基于数组的观测值重塑为指定的形状。这可以用于调整观测值的维度，以适应特定模型或处理流程的需要。

### 示例代码

```python
import gymnasium as gym
from gymnasium.wrappers import ReshapeObservation

# 创建环境并查看原始观测空间的形状
env = gym.make("CarRacing-v2")
print("Original observation space shape:", env.observation_space.shape)

# 应用 ReshapeObservation 包装器以改变观测值的形状
reshape_env = ReshapeObservation(env, (24, 4, 96, 1, 3))
print("Reshaped observation space shape:", reshape_env.observation_space.shape)

# 重置环境并获取重塑后的观测值
obs, _ = reshape_env.reset()
print("Shape of reshaped observation:", obs.shape)
```

### 参数解读
- **env** (`Env`): 要包装的环境实例。
- **shape** (`int | tuple[int, ...]`): 期望的新形状。可以是一个整数或一个整数的元组，表示新的观测数组维度。

### 注意事项
- `shape` 参数定义了观测值的新形状，需要确保重塑后的形状与原始数据中的总元素数相匹配。例如，如果原始观测值的形状是 `(96, 96, 3)`（即总共有 27648 个元素），则重塑形状 `(24, 4, 96, 1, 3)` 也应该包含相同数量的元素。
- 此包装器特别适用于需要调整观测数据形状以适应特定神经网络架构的场景。

### class gymnasium.wrappers.RescaleObservation(env: gym.Env[ObsType, ActType], min_obs: np.floating | np.integer | np.ndarray, max_obs: np.floating | np.integer | np.ndarray)

`gymnasium.wrappers.RescaleObservation` 包装器允许您对环境中的观测空间进行仿射（线性）缩放，以便观测值位于 `[min_obs, max_obs]` 范围内。这可以用来标准化观测值，使其适应模型的输入要求或提高学习算法的效率。

### 示例代码

```python
import gymnasium as gym
from gymnasium.wrappers import RescaleObservation
import numpy as np

# 创建环境并查看原始观测空间
env = gym.make("Pendulum-v1")
print("Original observation space:", env.observation_space)

# 应用 RescaleObservation 包装器以改变观测空间的范围
env = RescaleObservation(env, np.array([-2, -1, -10], dtype=np.float32), np.array([1, 0, 1], dtype=np.float32))
print("Rescaled observation space:", env.observation_space)

# 重置环境并获取重塑后的观测值
obs, _ = env.reset()
print("Rescaled observation:", obs)
```

### 参数解读
- **env** (`Env`): 要包装的环境实例。
- **min_obs** (`np.floating | np.integer | np.ndarray`): 新的最小观测界限。可以是标量或与环境观测空间形状相同的数组。
- **max_obs** (`np.floating | np.integer | np.ndarray`): 新的最大观测界限。同样，可以是标量或与环境观测空间形状相同的数组。

### 注意事项
- 此包装器特别适用于需要调整观测数据范围以适应特定神经网络架构的场景。
- 在设置 `min_obs` 和 `max_obs` 时，确保其形状与环境的观测空间相匹配，或者使用标量值来统一调整所有维度。
- 通过重新缩放观测值，可以帮助改进学习算法的性能，特别是在观测值的原始范围很大或非常不均匀时。

### class gymnasium.wrappers.TimeAwareObservation(env: Env[ObsType, ActType], flatten: bool = True, normalize_time: bool = False, *, dict_time_key: str = 'time')

`gymnasium.wrappers.TimeAwareObservation` 包装器通过增加步骤计数来扩展观测空间，以便在每个观测中包含当前在一集中进行的步数信息。这可以帮助代理了解它在一集中的进度，有时可以改善学习效果。

### 示例代码

```python
import gymnasium as gym
from gymnasium.wrappers import TimeAwareObservation

# 创建环境并应用 TimeAwareObservation 包装器
env = gym.make("CartPole-v1")
env = TimeAwareObservation(env, flatten=False, normalize_time=True)

# 查看扩展后的观测空间
print("Observation space after TimeAwareObservation:", env.observation_space)

# 重置环境并获取观测值
obs, _ = env.reset(seed=42)
print("Initial observation with time:", obs)

# 执行一个动作并获取新的观测值
obs, _, _, _, _ = env.step(env.action_space.sample())
print("Observation with time after one step:", obs)
```

### 参数解读
- **env** (`Env`): 要应用包装器的环境。
- **flatten** (`bool`): 如果设置为True，则将观测值展平为单一维度的Box。默认值为True。
- **normalize_time** (`bool`): 如果设置为True，则时间返回值在[0,1]范围内，代表距离截断前的剩余时间步数的归一化值。如果为False，则返回时间为整数步数。默认值为False。
- **dict_time_key** (`str`): 对于具有Dict观测空间的环境，时间空间的键值。默认为“time”。

### 注意事项
- 此包装器可以提高代理的时间感知能力，尤其是在需要考虑一集中的进度时。
- 如果环境已经具有`Dict`类型的观测空间，时间信息将作为新的键值对加入；否则，观测空间将转换为包含原始观测和时间信息的`Dict`。
- 使用`flatten`参数可以将观测空间简化为一维数组，这可能有助于某些类型的学习模型处理观测数据。
- 通过`normalize_time`参数，您可以选择将时间信息归一化，以便于模型处理不同长度集的统一表示。

# Reward Wrappers

### class gymnasium.RewardWrapper(env: Env[ObsType, ActType])


`gymnasium.RewardWrapper` 类为开发者提供了一种简便的方法来修改环境在每一步骤后返回的奖励值。通过继承这个类并重写 `reward` 方法，可以根据特定的需求调整奖励机制，这对于奖励形状的设计（reward shaping）来说非常有用，因为它允许开发者引入额外的奖励逻辑来帮助代理更快地学习或者优化其学习策略。

### 示例代码

下面的示例演示了如何创建一个自定义奖励包装器，该包装器将环境返回的原始奖励乘以一个固定的因子：

```python
import gymnasium as gym
from gymnasium import Env
from gymnasium.wrappers import RewardWrapper

class MultiplyReward(RewardWrapper):
    def __init__(self, env: Env, multiplier: float):
        super().__init__(env)
        self.multiplier = multiplier
    
    def reward(self, reward):
        # 将环境返回的奖励乘以指定的因子
        return reward * self.multiplier

# 创建环境实例
env = gym.make('CartPole-v1')

# 应用自定义的奖励包装器，将所有奖励乘以2
wrapped_env = MultiplyReward(env, multiplier=2.0)

# 使用包装后的环境
obs = wrapped_env.reset()
done = False
while not done:
    action = wrapped_env.action_space.sample()  # 随机选择动作
    obs, reward, done, _ = wrapped_env.step(action)
    print(f"Modified reward: {reward}")
```

### 参数解读
- **env** (`Env`): 要被包装的环境实例。
- **reward** (`SupportsFloat`): 环境在一步操作后返回的原始奖励值。

### `reward` 方法
- 输入参数 `reward` 是环境在一步操作后返回的原始奖励值。
- 返回值是修改后的奖励值，可以根据实际需要进行任意形式的转换，例如放大、缩小、加固定值等。

## Implemented Wrappers

### class gymnasium.wrappers.TransformReward(env: Env[ObsType, ActType], func: Callable[[SupportsFloat], SupportsFloat])


`gymnasium.wrappers.TransformReward` 类允许开发者对环境在每一步操作后返回的奖励值进行自定义变换。通过这种方式，可以根据需要调整奖励机制，比如引入额外的奖励逻辑或对奖励值进行缩放、平移等操作，以帮助代理学习更有效的策略。

### 示例代码

下面的代码示例展示了如何使用 `TransformReward` 包装器将所有奖励值乘以2再加1：

```python
import gymnasium as gym
from gymnasium.wrappers import TransformReward

# 定义一个函数，对奖励进行变换
def modify_reward(reward):
    return 2 * reward + 1

# 创建环境实例
env = gym.make("CartPole-v1")

# 应用TransformReward包装器，传入自定义的奖励变换函数
wrapped_env = TransformReward(env, func=modify_reward)

# 重置环境并进行一步动作
obs = wrapped_env.reset()
action = wrapped_env.action_space.sample()  # 随机选择动作
_, reward, _, _ = wrapped_env.step(action)

print(f"Modified reward: {reward}")
```

### 参数解读
- **env** (`Env`): 要被包装的环境实例。
- **func** (`Callable[[SupportsFloat], SupportsFloat]`): 应用于每一步返回奖励的函数。这个函数接受一个浮点数（原始奖励值）作为输入，并返回一个浮点数（变换后的奖励值）作为输出。

`TransformReward` 包装器通过将 `func` 函数应用于环境的每一步奖励，使得奖励值的变换变得简单而灵活，为奖励设计提供了广泛的可能性。这可以是非常有用的，特别是在需要对原始奖励进行调整以提高学习效率或改进策略性能的场景中。

### class gymnasium.wrappers.NormalizeReward(env: Env[ObsType, ActType], gamma: float = 0.99, epsilon: float = 1e-8)

`gymnasium.wrappers.NormalizeReward` 类用于标准化环境返回的即时奖励，使得奖励的指数移动平均具有固定的方差。这种处理方式旨在帮助学习算法更稳定和有效地学习，尤其是在奖励尺度变化大的环境中。

### 示例代码

以下代码示例演示如何使用 `NormalizeReward` 包装器来标准化奖励：

```python
import numpy as np
import gymnasium as gym
from gymnasium.wrappers import NormalizeReward

# 创建环境实例并应用NormalizeReward包装器
env = gym.make("MountainCarContinuous-v0")
wrapped_env = NormalizeReward(env, gamma=0.99, epsilon=1e-8)

# 重置环境并开始一个新的episode
_ = wrapped_env.reset(seed=123)
_ = wrapped_env.action_space.seed(123)

# 初始化奖励记录器
episode_rewards = []
terminated, truncated = False, False

# 运行一个episode，收集并记录奖励
while not (terminated or truncated):
    _, reward, terminated, truncated, _ = wrapped_env.step(wrapped_env.action_space.sample())
    episode_rewards.append(reward)

# 计算并打印标准化后的奖励方差
print("Variance of normalized rewards:", np.var(episode_rewards))
```

### 参数解读
- **env** (`Env`): 要被包装的环境实例。
- **gamma** (`float`): 在指数移动平均中使用的折扣因子。默认值为 `0.99`。
- **epsilon** (`float`): 稳定性参数，用于防止除以零的情况。默认值为 `1e-8`。

使用 `NormalizeReward` 包装器可以帮助处理奖励值跨度很大或分布不均的问题，通过将奖励标准化到一个较小的范围内，有助于提高学习算法的稳定性和性能。在实际使用时，可能需要根据具体任务调整 `gamma` 和 `epsilon` 参数以达到最佳效果。

### class gymnasium.wrappers.ClipReward(env: gym.Env[ObsType, ActType], min_reward: float | np.ndarray | None = None, max_reward: float | np.ndarray | None = None)


`gymnasium.wrappers.ClipReward` 类用于将环境返回的奖励裁剪到指定的上下界之间。这个包装器在奖励值范围过大或希望将奖励限制在特定范围内时非常有用，有助于改善学习算法的稳定性和性能。

### 示例代码

以下代码示例展示了如何使用 `ClipReward` 包装器来裁剪奖励：

```python
import gymnasium as gym
from gymnasium.wrappers import ClipReward

# 创建环境实例并应用ClipReward包装器
env = gym.make("CartPole-v1")
wrapped_env = ClipReward(env, min_reward=0, max_reward=0.5)

# 重置环境并开始一个新的episode
_ = wrapped_env.reset()

# 执行一个动作，并观察裁剪后的奖励值
_, reward, _, _, _ = wrapped_env.step(1)

# 打印裁剪后的奖励值
print("Clipped reward:", reward)
```

### 参数解读
- **env** (`Env`): 要被包装的环境实例。
- **min_reward** (`float` 或 `np.ndarray`): 奖励的下界。如果设置为 `None`，则不应用下界裁剪。
- **max_reward** (`float` 或 `np.ndarray`): 奖励的上界。如果设置为 `None`，则不应用上界裁剪。

使用 `ClipReward` 包装器可以帮助控制学习过程中奖励的范围，特别是在奖励的大小直接影响学习过程的环境中。通过限制奖励的范围，可以防止学习算法因为极端奖励值而出现不稳定的情况。在实际使用时，可能需要根据具体任务调整 `min_reward` 和 `max_reward` 参数以达到最佳学习效果。